In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

2023-05-18 14:08:41.197283: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("training_set.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [3]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,5,"In this memoir of Narciso Rodriguez, @PERSON3'...",5
1,2,5,Throughout the excerpt from Home the Blueprint...,5
2,3,5,The mood the author created in the memoir is l...,8
3,4,5,The mood created by the author is showing how ...,2
4,5,5,The mood created in the memoir is happiness an...,8


In [4]:
df['essay'][0]

"In this memoir of Narciso Rodriguez, @PERSON3's life, the mood containing it all, was greatful, and showed how his parents gave him love through his whole life. Both his parents were born and raised in Cuba, and in 1956, they both moved to the United States. Starting their lives over again, and taking any job they could find. Then in 1961, @PERSON2, @CAPS1. was born. Both Parents raised him with love, and care, and introducing his Cuban background into his life.        As @PERSON2, was telling his story, all readers could tell how greatful he was to have a family like he did. Also, as he shared his non-â\x80\x93 blood related family, and the remembrence on how his parents' life changed by moving to @LOCATION2, he showed the respect he had towards them."

In [5]:
temp.head()

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,5,"In this memoir of Narciso Rodriguez, life, the...",5,In memoir Narciso Rodriguez life mood contai...,547,127,7,4.307087,5,29,28,4,3
1,2,5,Throughout the excerpt from Home the Blueprint...,5,Throughout excerpt Home Blueprints Our Lives ...,720,168,7,4.285714,11,37,31,14,12
2,3,5,The mood the author created in the memoir is l...,8,The mood author created memoir love author fi...,500,110,6,4.545455,6,34,19,9,4
3,4,5,The mood created by the author is showing how ...,2,The mood created author showing cuban s lived ...,307,73,3,4.205479,8,16,15,8,1
4,5,5,The mood created in the memoir is happiness an...,8,The mood created memoir happiness gratitude N...,535,126,8,4.246032,4,28,22,11,9


In [6]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
X_train.shape

(1263, 3)

**PREPROCESSING**

In [9]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [10]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)


In [11]:
len(train_sents)

8407

In [12]:
train_sents[0]

['The', 'mood', 'created', 'author', 'memoir', 'sense', 'family']

**Preparing WORD2VEC and LSTM Model**

In [13]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [14]:
#Training Word2Vec model
num_features = 300 
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

model = Word2Vec(train_sents, 
                 workers=num_workers, 
                 vector_size=num_features, 
                 min_count = min_word_count, 
                 window = context, 
                 sample = downsampling)

model.init_sims(replace=True)
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)

/var/folders/kn/gnm3d9hn21d8t3q08syhwsdw0000gp/T/ipykernel_6778/1462232201.py:15: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [15]:
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,),dtype="float32")
    noOfWords = 0.
    index2word_set = set(model.wv.index_to_key)
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec,model.wv.get_vector(i))        
    vec = np.divide(vec,noOfWords)
    return vec


def getVecs(essays, model, num_features):
    c=0
    essay_vecs = np.zeros((len(essays),num_features),dtype="float32")
    for i in essays:
        essay_vecs[c] = makeVec(i, model, num_features)
        c+=1
    return essay_vecs


clean_train=[]
for i in train_e:
    clean_train.append(sent2word(i))
training_vectors = getVecs(clean_train, model, num_features)

clean_test=[] 

for i in test_e:
    clean_test.append(sent2word(i))
testing_vectors = getVecs(clean_test, model, num_features)

In [16]:
training_vectors.shape

(1263, 300)

In [17]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

2023-05-18 14:09:14.010785: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 300)            721200    
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 814,705
Trainable params: 814,705
Non-trainable params: 0
_________________________________________________________________


In [18]:
training_vectors.shape

(1263, 1, 300)

**TRAINING AND PREDICTION**

In [19]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
20/20 [==============================] - 5s 13ms/step - loss: 29.5296 - mae: 4.6946
Epoch 2/150
20/20 [==============================] - 0s 13ms/step - loss: 8.9859 - mae: 2.5051
Epoch 3/150
20/20 [==============================] - 0s 13ms/step - loss: 8.0843 - mae: 2.3799
Epoch 4/150
20/20 [==============================] - 0s 13ms/step - loss: 7.6926 - mae: 2.3429
Epoch 5/150
20/20 [==============================] - 0s 14ms/step - loss: 7.6804 - mae: 2.3178
Epoch 6/150
20/20 [==============================] - 0s 13ms/step - loss: 6.9927 - mae: 2.2084
Epoch 7/150
20/20 [==============================] - 0s 13ms/step - loss: 6.9867 - mae: 2.2178
Epoch 8/150
20/20 [==============================] - 0s 13ms/step - loss: 6.6904 - mae: 2.1876
Epoch 9/150
20/20 [==============================] - 0s 14ms/step - loss: 6.8162 - mae: 2.1822
Epoch 10/150
20/20 [==============================] - 0s 13ms/step - loss: 6.7224 - mae: 2.1551
Epoch 11/150
20/20 [============================

In [20]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

17/17 [==============================] - 1s 5ms/step


array([[8.],
       [6.],
       [4.],
       [7.],
       [7.],
       [7.],
       [7.],
       [7.],
       [7.],
       [7.],
       [7.],
       [6.],
       [7.],
       [7.],
       [7.],
       [7.],
       [6.],
       [8.],
       [5.],
       [6.],
       [7.],
       [6.],
       [7.],
       [7.],
       [7.],
       [5.],
       [6.],
       [7.],
       [8.],
       [7.],
       [7.],
       [7.],
       [7.],
       [6.],
       [6.],
       [7.],
       [7.],
       [6.],
       [7.],
       [7.],
       [6.],
       [7.],
       [7.],
       [7.],
       [6.],
       [4.],
       [8.],
       [5.],
       [7.],
       [7.],
       [6.],
       [6.],
       [7.],
       [5.],
       [6.],
       [4.],
       [7.],
       [5.],
       [7.],
       [7.],
       [7.],
       [5.],
       [7.],
       [7.],
       [7.],
       [6.],
       [7.],
       [5.],
       [6.],
       [7.],
       [5.],
       [7.],
       [7.],
       [6.],
       [7.],
       [5.],
       [6.],